In [1]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, DynamoDbDriver, ddb, _tr, Timestamp
from mand.core import RefData, RefDataUpdateEvent, Workbook, PrintMonitor

rawdb = DynamoDbDriver(ddb)
db = ObjectDb(rawdb)

_tr.RootClock('Main', db=db).write()
_tr.Clock('Portfolio', db=db).write()

In [2]:
class Book(Workbook):
    
    def books(self):
        return [self]

class PortfolioUpdateEvent(RefDataUpdateEvent):

    @node(stored=True)
    def children(self):
        return []
    
class Portfolio(RefData):

    evCls = PortfolioUpdateEvent
    
    @node
    def clock(self):
        return _tr.Clock.get('Portfolio', db=self.meta.db)
    
    @node
    def children(self):
        evs = self.activeEvents()
        if evs:
            return evs[-1].children()
        else:
            return []
        
    def setChildren(self, children, validTime=None, amends=[]):
        ev = self.evCls(entity=self, amends=amends, children=children, db=self.meta.db)
        ev.write(validTime=validTime)
        return ev
    
    def books(self):
        books = set()
        for c in self.children():
            for b in c.books():
                books.add(b)
        return list(books)

_tr.add(Book)
_tr.add(Portfolio)
_tr.add(PortfolioUpdateEvent)

In [3]:
with PrintMonitor():
    
    with db:
        checking = Book('checking').write()
        savings = Book('savings').write()
        brokerage = Book('brokerage').write()
        misc = Book('misc').write()
        margin = Book('margin').write()
        retirement = Book('401K').write()
        kid1Trust = Book('trust1').write()
        kid2Trust = Book('trust2').write()

        pAll = Portfolio('Family').write()
        pKids = Portfolio('Kids').write()
        pBanking = Portfolio('Banking').write()
        pTrading = Portfolio('Trading').write()
        pDerivs = Portfolio('Derivs').write()

    def info():
        c = [ o.meta.name() for o in pAll.children() ]
        b = [ o.meta.name() for o in pAll.books() ]
        print 'All children:', c
        print 'All books   :', b

    info()

    pKids.setChildren([kid1Trust, kid2Trust])
    pBanking.setChildren([checking, savings])
    pTrading.setChildren([brokerage, margin, retirement])
    pDerivs.setChildren([pTrading])
    pAll.setChildren([pBanking, pTrading, pKids])


    info()
    ts = Timestamp()

    pDerivs.setChildren([pTrading, misc])

    info()

 DB write /Global/Book/checking
 DB write /Global/Book/savings
 DB write /Global/Book/brokerage
 DB write /Global/Book/misc
 DB write /Global/Book/margin
 DB write /Global/Book/401K
 DB write /Global/Book/trust1
 DB write /Global/Book/trust2
 DB write /Global/Portfolio/Family
 DB write /Global/Portfolio/Kids
 DB write /Global/Portfolio/Banking
 DB write /Global/Portfolio/Trading
 DB write /Global/Portfolio/Derivs
 GetValue begin </Global/Portfolio/Family>.children()
   GetValue/Calc begin Portfolio:children
     GetValue begin </Global/Portfolio/Family>.clock()
       GetValue/Calc begin Portfolio:clock
     GetValue begin </Global/Clock/Portfolio>.cutoffs()
       GetValue/Calc begin Clock:cutoffs
         GetValue begin </Global/Clock/Portfolio>.parent()
           GetValue/Calc begin Clock:parent
             GetValue begin </Global/Clock/Portfolio>.clock()
               GetValue/Calc begin Entity:clock
             GetValue begin </Global/RootClock/Main>.cutoffs()
               G